# Model Selection and Tuning

This file is concerned with trying different ML algorithms for classification, focusing exclusively on predictive power and starting from the simplest model to the most complicated. We will use the processed data created from the data_preparation.ipynb file, which is a csv named 'train_data_processed.csv'.

The objective of the models is to accurately predict if a client with certain characteristics will default on a loan when asking for it. As we do not know the associated costs with a client not paying back nor the benefits of a client repaying, we will focus on Area Under Curve as the performance metric.

Every model will be trained, tuned and cross-validated using the sci-kit learn library.

In [8]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

jobs = os.cpu_count()-1 ## A lot of power.

In [9]:
df = pd.read_csv('train_data_processed.csv', index_col= 0)
df.head()

,AMT_ANNUITY,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,CNT_CHILDREN,AMT_INCOME_TOTAL,...,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,REGION_POPULATION_RELATIVE,TARGET
0,-0.166149,0.241033,4.163149,0.250314,5.253007,0.083037,0.262949,0.139376,-0.577538,0.142129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018801,1
1,0.592677,-0.176156,-0.321603,-0.170589,-0.276616,0.311267,0.622246,0.510853,-0.577538,0.426792,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003541,0
2,-1.404676,-0.593345,-0.321603,-0.591491,-0.276616,0.502130,0.555912,0.729567,-0.577538,-0.427196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010032,0
3,0.177869,0.241033,-0.321603,0.250314,-0.276616,0.502130,0.650442,0.510853,-0.577538,-0.142533,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008019,0
4,-0.361755,-0.593345,-0.321603,-0.591491,-0.276616,0.502130,0.322738,0.510853,-0.577538,-0.199466,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028663,0


The data is already scaled for numerical features and encoded (One-hot) for categorical features. We are ready to start working with it.

In [10]:
df.rename(columns = {col: col.lower() for col in df.columns.values}, inplace = True)
X = df.drop(columns=['target'])
y = df.target.copy()

performance_metrics = {}

### First Model: **Logistic Regression**

For this model we will first fit a logistic regression to all the parameters, and take the mean of the AUC score from a 10 fold cross validation. We will then run a logistic regression regularized by Lasso to try and do feature extraction. We will finally compute the regression without the penalty but using only the parameters extracted from the constrained regression.

In [11]:
from sklearn.linear_model import SGDClassifier

In [12]:
start = time.time()
logistic_regression_full = SGDClassifier(loss = 'log_loss', l1_ratio= 1, n_jobs= jobs, random_state= 0, early_stopping= True)
results_logistic_regression_full = cross_val_score(logistic_regression_full, X,y, cv = 10, scoring='roc_auc')
end = time.time()
performance_metrics['logistic_regression_full'] = {'error' : np.mean(results_logistic_regression_full),'runtime': end-start}

In [13]:
performance_metrics['logistic_regression_full']

{'logistic_regression_full': {'error': 0.7363660142084504,
  'runtime': 36.39678478240967}}

Regularized Logistic regression

In [16]:
param_grid = [{'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}]
logistic_regression_l1 = SGDClassifier(loss = 'log_loss', l1_ratio= 1, n_jobs= jobs, random_state= 0, early_stopping= True)
grid_search = GridSearchCV(logistic_regression_l1, param_grid, cv = 10, scoring = 'roc_auc', return_train_score= True)
grid_search.fit(X,y)
grid_search.best_params_

SGDClassifier(early_stopping=True, l1_ratio=1, loss='log_loss', n_jobs=7,
              random_state=0)

As the optimal parameter happens to be the default one, with little to no regularization, we will not remove features using Lasso regression and will accept the original logistic regression as our best model for it.